In [1]:
#import
import pandas as pd
import numpy as np

In [2]:
#read in csv files
student_file = "Resources/students_complete.csv"
school_file = "Resources/schools_complete.csv"

#create data frames
students = pd.read_csv(student_file)
schools = pd.read_csv(school_file)

In [3]:
#print column names to find the variable to merge on
#print(students.columns)
#print(schools.columns)

In [4]:
#merge data frames
#using left merge because I want the school information for each individual student
district = pd.merge(students, schools, on="school_name", how="left")

In [5]:
#Adding additional columns for percent passing (see Notes.txt)
district["pass_reading"] = [1 if x > 69 else 0 for x in district["reading_score"]]
district["pass_math"] = [1 if x > 69 else 0 for x in district["math_score"]]

In [6]:
#Creating District Summary TAble 
#Part I: calculating metrics for table

#calculate total schools
total_schools = schools["school_name"].nunique()

#calculate total students
total_students = schools["size"].sum()

#calculate total budget 
total_budget = schools["budget"].sum()

#calculate average math score
avg_math = students["math_score"].mean()

#calculate average reading score
avg_read = students["reading_score"].mean()

#calculate percent passing math (see Notes.txt)
math_df = students.loc[students["math_score"] > 69, :]
math_pass = math_df["Student ID"].nunique()
math_pass_rate = (math_pass / total_students) * 100

#calculate percent passing reading (see Notes.txt)
read_df = students.loc[students["reading_score"] > 69, :]
read_pass = read_df["Student ID"].nunique()
read_pass_rate = (read_pass / total_students) * 100

#calculate overall passing rate (see Notes.txt)
overall_rate = (read_pass_rate + math_pass_rate)/2

In [7]:
#Part II: Creating summary table and formating it
#create District Summary table
district_summary = pd.DataFrame({"Total Schools": [total_schools],
                                "Total Students": [total_students],
                                "Total Budget": [total_budget],
                                "Average Math Score": [avg_math],
                                "Average Reading Score": [avg_read],
                                "Percent Passing Math": [math_pass_rate],
                                "Percent Passing Reading": [read_pass_rate],
                                "Overall Passing Rate": [overall_rate]})
#format District Summary Table
district_summary["Total Students"]=district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"]=district_summary["Total Budget"].map("${:,.2f}".format)
district_summary["Percent Passing Math"]=district_summary["Percent Passing Math"].map("{:.2f}%".format)
district_summary["Percent Passing Reading"]=district_summary["Percent Passing Reading"].map("{:.2f}%".format)
district_summary["Overall Passing Rate"]=district_summary["Overall Passing Rate"].map("{:.2f}%".format)

## District Summary Table

In [8]:
#Print District Summary Table
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.98%,85.81%,80.39%


In [9]:
#Creating the School Summary table metrics

#calculate school name, total students, average math score, average reading score, percent passing math, percent passing reading
stu_sum = district.groupby(["school_name"], as_index=False).agg({
    "Student ID": "count",
    "reading_score": "mean",
    "math_score": "mean",
    "pass_reading": "mean",
    "pass_math": "mean"
})

#Pull school type and School budget from original table
summary_student = pd.merge(stu_sum, schools[["school_name", "type", "budget"]], on="school_name", how="left")

#caluclate per student bugdet
summary_student["Per Student Budget"] = summary_student["budget"] / summary_student["Student ID"]

#caluclate percent overall passing rate
summary_student["Overall Passing Rate"] = (summary_student["pass_reading"] + summary_student["pass_math"]) / 2 *100

#Multiplying percent passsing by 100
summary_student["pass_reading"] = summary_student["pass_reading"] * 100
summary_student["pass_math"] = summary_student["pass_math"] * 100

In [10]:
#Re-name columns:
summary_student = summary_student.rename(columns={
    "school_name": "School Name",
    "Student ID": "Total Students",
    "reading_score": "Average Reading Score",
    "math_score": "Average Math Score",
    "pass_reading": "Percent Passing Reading",
    "pass_math": "Percent Passing Math",
    "type": "School Type",
    "budget": "Total School Budget"
})

#Re-arrange columns
student_summary = summary_student[["School Name", "School Type", "Total Students",
                                  "Total School Budget", "Per Student Budget", "Average Math Score",
                                  "Average Reading Score", "Percent Passing Math", 
                                  "Percent Passing Reading", "Overall Passing Rate"]]

#Save a copy before formatting
student_summary_final = student_summary.copy()

#format School Summary Table
student_summary_final["Total Students"]=student_summary_final["Total Students"].map("{:,}".format)
student_summary_final["Total School Budget"]=student_summary_final["Total School Budget"].map("${:,.2f}".format)
student_summary_final["Per Student Budget"]=student_summary_final["Per Student Budget"].map("${:,.2f}".format)
student_summary_final["Percent Passing Math"]=student_summary_final["Percent Passing Math"].map("{:.2f}%".format)
student_summary_final["Percent Passing Reading"]=student_summary_final["Percent Passing Reading"].map("{:.2f}%".format)
student_summary_final["Overall Passing Rate"]=student_summary_final["Overall Passing Rate"].map("{:.2f}%".format)

## School Summary Table

In [11]:
#Print School Summary Table
student_summary_final

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
0,Bailey High School,District,"4,976","$3,124,928.00",$628.00,77.048432,81.033963,66.68%,81.93%,74.31%
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,95.59%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,73.36%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,73.80%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,95.27%
5,Hernandez High School,District,"4,635","$3,022,020.00",$652.00,77.289752,80.934412,66.75%,80.86%,73.81%
6,Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.51%,96.25%,94.38%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,73.50%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,73.64%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,95.27%


In [12]:
#Top Performing Schools (by passing rate)
#Top 5 Schools
#Sort the Summary table by overall passing rate
top_5 = student_summary.sort_values(["Overall Passing Rate"], ascending=False)

#find the 5 largest values of overall passing rate
top_5_summary = top_5.nlargest(5, "Overall Passing Rate")

#format Top 5 Schools Summary Table
top_5_summary["Total Students"]=top_5_summary["Total Students"].map("{:,}".format)
top_5_summary["Total School Budget"]=top_5_summary["Total School Budget"].map("${:,.2f}".format)
top_5_summary["Per Student Budget"]=top_5_summary["Per Student Budget"].map("${:,.2f}".format)
top_5_summary["Percent Passing Math"]=top_5_summary["Percent Passing Math"].map("{:.2f}%".format)
top_5_summary["Percent Passing Reading"]=top_5_summary["Percent Passing Reading"].map("{:.2f}%".format)
top_5_summary["Overall Passing Rate"]=top_5_summary["Overall Passing Rate"].map("{:.2f}%".format)

## Top 5 Schools (By Overall Passing Rate)

In [13]:
#Print Table
top_5_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
1,Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.13%,97.04%,95.59%
12,Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.27%,97.31%,95.29%
9,Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.59%,95.95%,95.27%
4,Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.39%,97.14%,95.27%
13,Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.87%,96.54%,95.20%


In [14]:
#Bottom Performing Schools (by passing rate)
#Bottom 5 Schools
#Sort the Summary table by overall passing rate
bottom_5 = student_summary.sort_values(["Overall Passing Rate"], ascending=True)

#find the 5 smallest values of overall passing rate
bottom_5_summary = bottom_5.nsmallest(5, "Overall Passing Rate")

#format Bottom 5 Schools Summary Table
bottom_5_summary["Total Students"]=bottom_5_summary["Total Students"].map("{:,}".format)
bottom_5_summary["Total School Budget"]=bottom_5_summary["Total School Budget"].map("${:,.2f}".format)
bottom_5_summary["Per Student Budget"]=bottom_5_summary["Per Student Budget"].map("${:,.2f}".format)
bottom_5_summary["Percent Passing Math"]=bottom_5_summary["Percent Passing Math"].map("{:.2f}%".format)
bottom_5_summary["Percent Passing Reading"]=bottom_5_summary["Percent Passing Reading"].map("{:.2f}%".format)
bottom_5_summary["Overall Passing Rate"]=bottom_5_summary["Overall Passing Rate"].map("{:.2f}%".format)


## Bottom 5 Schools (by Overall Passing Rate)

In [15]:
#Print Table
bottom_5_summary

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
10,Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.37%,80.22%,73.29%
2,Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.99%,80.74%,73.36%
7,Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.68%,81.32%,73.50%
8,Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.06%,81.22%,73.64%
3,Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.31%,79.30%,73.80%


In [16]:
#Math Scores by Grade
#average math score for each grade at each school

#calculate school name, average math score by grade
grade_9math = district[district["grade"] =="9th"].groupby(["school_name", "grade"], as_index=False).agg({"math_score": "mean"})
grade_10math = district[district["grade"] =="10th"].groupby(["school_name", "grade"], as_index=False).agg({"math_score": "mean"})
grade_11math = district[district["grade"] =="11th"].groupby(["school_name", "grade"], as_index=False).agg({"math_score": "mean"})
grade_12math = district[district["grade"] =="12th"].groupby(["school_name", "grade"], as_index=False).agg({"math_score": "mean"})

#Merge Data Sets into One
avgmath_grade = pd.merge(grade_9math, grade_10math[["school_name", "math_score"]], on="school_name", how="left",
                        suffixes=('_9th', '_10th'))
avgmath_grade = pd.merge(avgmath_grade, grade_11math[["school_name", "math_score"]], on="school_name", how="left")
avgmath_grade = pd.merge(avgmath_grade, grade_12math[["school_name", "math_score"]], on="school_name", how="left",
                         suffixes=('_11th', '_12th'))
avgmath_grade

#Rename columns:
avgmath_grade = avgmath_grade.rename(columns={
    "school_name": "School Name",
    "math_score_9th": "9th Grade",
    "math_score_10th": "10th Grade",
    "math_score_11th": "11th Grade",
    "math_score_12th": "12th Grade"
})
avgmath_grade

#delete grade column
avgmathbygrade = avgmath_grade.drop(columns="grade")

## Average Math Score by Grade

In [17]:
#Print average math summary table
avgmathbygrade

,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,77.083676,76.996772,77.515588,76.492218
1,Cabrera High School,83.094697,83.154506,82.765560,83.277487
2,Figueroa High School,76.403037,76.539974,76.884344,77.151369
3,Ford High School,77.361345,77.672316,76.918058,76.179963
4,Griffin High School,82.044010,84.229064,83.842105,83.356164
5,Hernandez High School,77.438495,77.337408,77.136029,77.186567
6,Holden High School,83.787402,83.429825,85.000000,82.855422
7,Huang High School,77.027251,75.908735,76.446602,77.225641
8,Johnson High School,77.187857,76.691117,77.491653,76.863248
9,Pena High School,83.625455,83.372000,84.328125,84.121547


In [18]:
#Reading scores by grade
#average reading score for each grade at each school

#calculate school name, average math score by grade
grade_9read = district[district["grade"] =="9th"].groupby(["school_name", "grade"], as_index=False).agg({"reading_score": "mean"})
grade_10read = district[district["grade"] =="10th"].groupby(["school_name", "grade"], as_index=False).agg({"reading_score": "mean"})
grade_11read = district[district["grade"] =="11th"].groupby(["school_name", "grade"], as_index=False).agg({"reading_score": "mean"})
grade_12read = district[district["grade"] =="12th"].groupby(["school_name", "grade"], as_index=False).agg({"reading_score": "mean"})

#Merge Data Sets into One
avgread_grade = pd.merge(grade_9read, grade_10read[["school_name", "reading_score"]], on="school_name", how="left",
                        suffixes=('_9th', '_10th'))
avgread_grade = pd.merge(avgread_grade, grade_11read[["school_name", "reading_score"]], on="school_name", how="left")
avgread_grade = pd.merge(avgread_grade, grade_12read[["school_name", "reading_score"]], on="school_name", how="left",
                         suffixes=('_11th', '_12th'))

#Rename columns:
avgread_grade = avgread_grade.rename(columns={
    "school_name": "School Name",
    "reading_score_9th": "9th Grade",
    "reading_score_10th": "10th Grade",
    "reading_score_11th": "11th Grade",
    "reading_score_12th": "12th Grade"
})

#delete grade column
avgreadbygrade = avgread_grade.drop(columns="grade")

## Average Reading Score by Grade

In [19]:
#Print average reading score by grade summary table
avgreadbygrade

,School Name,9th Grade,10th Grade,11th Grade,12th Grade
0,Bailey High School,81.303155,80.907183,80.945643,80.912451
1,Cabrera High School,83.676136,84.253219,83.788382,84.287958
2,Figueroa High School,81.198598,81.408912,80.640339,81.384863
3,Ford High School,80.632653,81.262712,80.403642,80.662338
4,Griffin High School,83.369193,83.706897,84.288089,84.013699
5,Hernandez High School,80.866860,80.660147,81.396140,80.857143
6,Holden High School,83.677165,83.324561,83.815534,84.698795
7,Huang High School,81.290284,81.512386,81.417476,80.305983
8,Johnson High School,81.260714,80.773431,80.616027,81.227564
9,Pena High School,83.807273,83.612000,84.335938,84.591160


In [20]:
#Scores by School Spending
#school performance based on average spending ranges (per student)

#create data frame
spending_summary = student_summary[["Per Student Budget", "Average Math Score", "Average Reading Score", 
                                    "Percent Passing Math", "Percent Passing Reading", "Overall Passing Rate"]].copy()

#creating bins
spending_bins = [0, 585, 615, 645, 675]
spending_labels = ["<$585", "$585-615", "$615-645", ">$675"]

#Cutting frame into bins
bin_cutting = pd.cut(spending_summary['Per Student Budget'], bins=spending_bins, labels=spending_labels)
spending_summary["Student Spending"] = bin_cutting

#Group by bins and print
student_spending = spending_summary.groupby("Student Spending")
perstudent_spending = pd.DataFrame(student_spending.mean())

#delete spending column
student_budget = perstudent_spending.drop(columns="Per Student Budget")

#Format Data Frame
student_budget["Percent Passing Math"]=student_budget["Percent Passing Math"].map("{:.2f}%". format)
student_budget["Percent Passing Reading"]=student_budget["Percent Passing Reading"].map("{:.2f}%". format)
student_budget["Overall Passing Rate"]=student_budget["Overall Passing Rate"].map("{:.2f}%". format)

## School Performance by Per Student Budget

In [21]:
#Print per student budget table
student_budget

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Student Spending,,,,,
<$585,83.455399,83.933814,93.46%,96.61%,95.04%
$585-615,83.599686,83.885211,94.23%,95.90%,95.07%
$615-645,79.079225,81.891436,75.67%,86.11%,80.89%
>$675,76.997210,81.027843,66.16%,81.13%,73.65%


In [22]:
#Scores by school size
#school performance based on school size
#use 3 resonable bins to group school size - small, medium, large
#Table should include: average math score, average reading score, percent
#passing math, percent passing reading, and overall percent passing rate

size_summary = student_summary[["Average Math Score", "Average Reading Score", "Percent Passing Math", "Percent Passing Reading", "Overall Passing Rate"]].copy()

#creating bins labels
size_bins = [0, 1000, 2000, 5000]
size_labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

#Cutting frame into bins
size_summary["Size Group"] = pd.cut(student_summary["Total Students"], bins=size_bins, labels=size_labels)

#Group by bins and pass to dataframe
school_size = size_summary.groupby("Size Group")
schoolbysize = pd.DataFrame(school_size.mean())

#Format Data Frame
schoolbysize["Percent Passing Math"]=schoolbysize["Percent Passing Math"].map("{:.2f}%". format)
schoolbysize["Percent Passing Reading"]=schoolbysize["Percent Passing Reading"].map("{:.2f}%". format)
schoolbysize["Overall Passing Rate"]=schoolbysize["Overall Passing Rate"].map("{:.2f}%". format)

## School Performance by School Size

In [23]:
#Print school size table
schoolbysize

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
Size Group,,,,,
Small (<1000),83.821598,83.929843,93.55%,96.10%,94.82%
Medium (1000-2000),83.374684,83.864438,93.60%,96.79%,95.20%
Large (2000-5000),77.746417,81.344493,69.96%,82.77%,76.36%


In [24]:
#Scores by school type
#school performance based on school type
#use 2 bins to group - district, charter
#Table should include: average math score, average reading score, percent
#passing math, percent passing reading, and overall percent passing rate

school_type = student_summary.groupby(["School Type"]).agg({
    "Average Math Score": "mean",
    "Average Reading Score": "mean",
    "Percent Passing Math": "mean",
    "Percent Passing Reading": "mean",
    "Overall Passing Rate": "mean"
})

#format School Type Summary Table
school_type["Percent Passing Math"]=school_type["Percent Passing Math"].map("{:.2f}%".format)
school_type["Percent Passing Reading"]=school_type["Percent Passing Reading"].map("{:.2f}%".format)
school_type["Overall Passing Rate"]=school_type["Overall Passing Rate"].map("{:.2f}%".format)

## School Performance by School Type

In [25]:
#Print School Summary Table
school_type

,Average Math Score,Average Reading Score,Percent Passing Math,Percent Passing Reading,Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.62%,96.59%,95.10%
District,76.956733,80.966636,66.55%,80.80%,73.67%
